# idact - Prometheus sandbox

## Initial setup

Add `idact` to path:

In [1]:
import sys
import os
import bitmath
import getpass
import contextlib
import fabric
import logging
from pprint import pprint

def append_idact_path():
    idact_path = os.path.realpath(os.path.join(os.getcwd(), '../'))
    sys.path.append(idact_path)
append_idact_path()

from idact import *
from idact.detail.auth.set_password import set_password

os.environ['IDACT_KEY_LOCATION'] = os.path.join(os.getcwd(), '../.notebook-ssh')
# os.environ['IDACT_KEY_LOCATION'] = os.path.expanduser('~/.ssh')
if not os.path.exists(os.environ['IDACT_KEY_LOCATION']):
    os.mkdir(os.environ['IDACT_KEY_LOCATION'])

USER = 'plggarstka'

Hide debug information, setup context manager stack (for testing purposes)

## Add cluster (only first run)

In [2]:
key = KeyType.RSA  # Generate RSA key
# key = os.path.join(os.path.expanduser('~/.ssh'), 'id_rsa')

In [3]:
cluster = add_cluster(name="pro",
                      user=USER,
                      host="pro.cyfronet.pl",
                      port=22,
                      auth=AuthMethod.PUBLIC_KEY,
                      key=key,
                      install_key=True,
                      scratch="$SCRATCH")
save_environment('.idact-env')

2018-10-14 23:30:19 INFO: Generating public-private key pair.


## Load cluster (subsequent runs)

In [4]:
load_environment('.idact-env')
cluster = show_cluster("pro")
cluster

Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_ut', install_key=True, disable_sshd=False)

In [5]:
set_log_level(logging.INFO)
#set_log_level(logging.DEBUG)
save_environment('.idact-env')

In [6]:
node = cluster.get_access_node()
node

Node(pro.cyfronet.pl:22, None)

On your first action, you will be asked for a password to install the key.
You can connect explicitly (optional) to do this right now:

In [7]:
node.connect()

2018-10-14 23:30:23 INFO: Installing key using password authentication.
Password for plggarstka@pro.cyfronet.pl:22: 


In [8]:
node.run('whoami')

'plggarstka'

In [9]:
node.run('hostname')

'login01.pro.cyfronet.pl'

## Allocate nodes

In [10]:
nodes = cluster.allocate_nodes(nodes=2,
                               cores=2,
                               memory_per_node=bitmath.GiB(10),
                               walltime=Walltime(minutes=20),
                               native_args={
                                   '--partition': 'plgrid-testing',
                                   '--account': 'intdata'
                               })

2018-10-14 23:30:49 INFO: Creating the ssh directory.


In [11]:
nodes

Nodes([Node(NotAllocated),Node(NotAllocated)], SlurmAllocation(job_id=13687014))

In [12]:
nodes.wait()
nodes

Nodes([Node(p1037:42730, 2018-10-14 21:50:57.071443+00:00),Node(p1038:39801, 2018-10-14 21:50:57.071443+00:00)], SlurmAllocation(job_id=13687014))

## Run commands

In [13]:
nodes[0].run('whoami')

'plggarstka'

In [14]:
nodes[0].run('hostname')

'p1037'

In [15]:
nodes[1].run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          13687014 plgrid-te     wrap plggarst  R       0:17      2 p[1037-1038]'

In [16]:
nodes[1].run('hostname')

'p1038'

## Examine node resources

In [17]:
nodes[0].resources.memory_total

GiB(10.0)

In [18]:
nodes[0].resources.memory_usage

GiB(0.022357940673828125)

In [19]:
nodes[0].resources.cpu_cores

2

In [20]:
nodes[0].resources.cpu_usage

1.0

## Tunnel

In [21]:
tunnel = nodes[0].tunnel(here=9000, there=10000)

In [22]:
tunnel

MultiHopTunnel(9000:10000)

In [23]:
tunnel.close()

## Deploy notebook

One-time config step (cluster-specific):

In [24]:
cluster.config.setup_actions.jupyter = ['module load plgrid/tools/python-intel/3.6.2']
save_environment('.idact-env')

To run Jupyter Notebook on the cluster:

In [25]:
nb = nodes[0].deploy_notebook(local_port=8080)
nb

2018-10-14 23:31:27 WARNING: Exception: Cannot run task., retry 1/5.


JupyterDeployment(8080 -> Node(p1037:42730, 2018-10-14 21:50:57.071443+00:00))

In [26]:
nodes[0].resources.memory_usage

GiB(0.08097076416015625)

In [27]:
nb.local_port

8080

To open the deployed notebook server in a new tab:

In [28]:
nb.open_in_browser()

In [29]:
nodes[0].resources.memory_usage

GiB(0.08096694946289062)

In [30]:
nb.cancel()

2018-10-14 23:31:33 INFO: Cancelling Jupyter deployment.


## Push and pull nodes

In order to work with Dask, you would usually need a notebook running on the cluster, as shown above.

To access the allocated nodes from the cluster, you need to push their deployment first:

In [31]:
cluster.push_deployment(nodes)

2018-10-14 23:31:40 INFO: Pushing deployment: Nodes([Node(p1037:42730, 2018-10-14 21:50:57.071443+00:00),Node(p1038:39801, 2018-10-14 21:50:57.071443+00:00)], SlurmAllocation(job_id=13687014))


Then, you would pull the deployment on the cluster:

In [32]:
deployments = cluster.pull_deployments()
deployments

2018-10-14 23:31:45 INFO: Pulling deployments.
2018-10-14 23:31:49 INFO: Pulled node deployment: Nodes([Node(p1037:42730, 2018-10-14 21:50:57.071443+00:00),Node(p1038:39801, 2018-10-14 21:50:57.071443+00:00)], SlurmAllocation(job_id=13687014))


SynchronizedDeployments(nodes=1)

In [33]:
nodes = deployments.nodes[0]
nodes

Nodes([Node(p1037:42730, 2018-10-14 21:50:57.071443+00:00),Node(p1038:39801, 2018-10-14 21:50:57.071443+00:00)], SlurmAllocation(job_id=13687014))

Essentially, this feature is intended for using an allocation in multiple notebooks at once.

Deployments are cleared automatically if they are expired or cancelled. They can also be cleared manually by  running:

In [34]:
cluster.clear_pushed_deployments()

2018-10-14 23:31:50 INFO: Clearing deployments.


## Deploy Dask

One-time config step (cluster-specific):

In [35]:
cluster.config.setup_actions.dask = ['module load plgrid/tools/python-intel/3.6.2']
cluster.config.scratch = '$SCRATCH'
save_environment('.idact-env')

In [36]:
dd = deploy_dask(nodes)
dd

2018-10-14 23:32:00 INFO: Deploying Dask on 2 nodes.
2018-10-14 23:32:00 INFO: Connecting to p1037:42730 (1/2).
2018-10-14 23:32:01 INFO: Connecting to p1038:39801 (2/2).
2018-10-14 23:32:01 INFO: Deploying scheduler on the first node: p1037.
2018-10-14 23:32:11 WARNING: Exception: Unable to determine scheduler address., retry 1/5.
2018-10-14 23:32:20 INFO: Checking scheduler connectivity from p1037 (1/2).
2018-10-14 23:32:20 INFO: Checking scheduler connectivity from p1038 (2/2).
2018-10-14 23:32:20 INFO: Deploying workers.
2018-10-14 23:32:20 INFO: Deploying worker 1/2.
2018-10-14 23:32:31 INFO: Deploying worker 2/2.
2018-10-14 23:32:41 WARNING: Exception: Cannot run task., retry 1/5.
2018-10-14 23:32:48 INFO: Validating worker 1/2.
2018-10-14 23:32:48 INFO: Validating worker 2/2.


DaskDeployment(scheduler=tcp://localhost:62082/tcp://172.20.68.17:38984, workers=2)

In [37]:
nodes[0].resources.memory_usage

GiB(0.3689422607421875)

Get Dask client:

In [38]:
client = dd.get_client()
client

<Client: scheduler='tcp://172.20.68.17:38984' processes=2 cores=4>

In [39]:
nodes[0].resources.cpu_usage

4.0

Computation will work only if Python and library versions match:

In [40]:
#x = client.submit(lambda: value + 1, 10)
#x.result() == 11

Diagnostics servers are tunnelled:

In [41]:
dd.diagnostics.addresses

['http://localhost:59943', 'http://localhost:62093', 'http://localhost:62101']

To open diagnostics servers in new tabs:

In [42]:
dd.diagnostics.open_all()

In [43]:
dd.cancel()

2018-10-14 23:32:51 INFO: Cancelling worker deployment on p1038.
2018-10-14 23:32:58 INFO: Cancelling worker deployment on p1037.
2018-10-14 23:33:05 INFO: Cancelling scheduler deployment on p1037.


## Close

In [44]:
nodes.running()

True

In [45]:
nodes.cancel()

2018-10-14 23:33:13 INFO: Cancelling job 13687014.


In [46]:
nodes.running()

False

In [47]:
node.run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          13687014 plgrid-te     wrap plggarst CG       2:16      2 p[1037-1038]'

## Push and pull the environment

When working on a cluster, it may be useful to synchronize idact config with the local machine. Pushing the environment will merge the local environment into the remote environment.

In [48]:
push_environment(cluster, path='~/.idact-notebook-env')

2018-10-14 23:33:16 INFO: Pushing the environment to cluster.
2018-10-14 23:33:19 ERROR: Failure: Getting file from node pro.cyfronet.pl: /net/people/plggarstka/.idact-notebook-env
2018-10-14 23:33:19 ERROR: Failure: Deserializing the environment from cluster.
2018-10-14 23:33:19 INFO: Remote environment is missing, current environment will be copied to cluster.


In [49]:
print(node.run('cat ~/.idact-notebook-env'))

{
    "clusters": {
        "pro": {
            "auth": "PUBLIC_KEY",
            "disableSshd": false,
            "host": "pro.cyfronet.pl",
            "installKey": true,
            "key": null,
            "port": 22,
            "portInfoRetries": 5,
            "scratch": "$SCRATCH",
            "setupActions": {
                "dask": [
                    "module load plgrid/tools/python-intel/3.6.2"
                ],
                "jupyter": [
                    "module load plgrid/tools/python-intel/3.6.2"
                ]
            },
            "user": "plggarstka"
        }
    },
    "logLevel": 20
}


The reverse operation is pulling the environment, which merges the remote environment into the local environment. Machine-specific information like the private key path is skipped when pushing or pulling.

In [50]:
pull_environment(cluster, path='~/.idact-notebook-env')

2018-10-14 23:33:23 INFO: Pulling the environment from cluster.


The 'path' parameter is optional. It defaults to ~/.idact.conf, or the value of the remote IDACT_CONFIG_PATH environment variable.

In [51]:
node.run('rm -v ~/.idact-notebook-env')

'removed ‘/net/people/plggarstka/.idact-notebook-env’'

## Remove cluster

A cluster can be removed from the environment.

In [52]:
add_cluster(name='fake',
            user='fakeuser',
            host='fakehost',
            port=2222)

2018-10-14 23:33:24 INFO: No auth method specified, defaulting to password-based.


Cluster(fakehost, 2222, fakeuser, auth=AuthMethod.ASK, key=None, install_key=True, disable_sshd=False)

In [53]:
show_clusters()

{'pro': Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_ut', install_key=False, disable_sshd=False),
 'fake': Cluster(fakehost, 2222, fakeuser, auth=AuthMethod.ASK, key=None, install_key=True, disable_sshd=False)}

In [54]:
remove_cluster('fake')

In [55]:
show_clusters()

{'pro': Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_ut', install_key=False, disable_sshd=False)}